# Reference Suitability
This notebook is a tool for determining which standard materials are suitable for use as a fitting reference for an element.  The problem it is designed to address is interferences between the element and other elements in the material. It considers a default list of pure elemental and stoichiometric compound standard materials.  The characteristic X-ray associated with the specified element are divided into regions determined by the detector performance each of which will be fit independently.  The algorithm then checks each material containing the element of interest to determine which of these regions does not suffer an interference with any of the other elements in the material.

In [1]:
# Load the necessary libraries
using NeXLSpectrum
using DataFrames
# Configure tabulation
ENV["lines"], ENV["columns"] = 200, 400;


The performance of your detector is critical for determining how the peaks will be split up into independent regions each of which is fit independently.

In [2]:
# Build a detector model to match your EDS detector
# BasicEDS(channelCount, zeroOffset, channelWidth, FWHM_MnKa, low_level_discriminator, visibility)
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
  Dict( # What is the first element with visible KShell, LShell, ... peaks
    KShell=>n"Be",
    LShell=>n"Ca",
    MShell=>n"Cs",
    NShell=>n"Pu")
)

BasicEDS[4096 chs, E[ch] = -480.40409 + 5.00525⋅ch, 132.0 eV @ Mn K-L3, 110 ch LLD, [Be,Ca,Cs,Pu]]

Specify an element and load a default set of standard materials.  You can add additional materials if desired.

In [3]:
elm = parse(Element, "Hg")
stds=getstandards(elm, 0.01)
# push!(stds, parse(Material, "0.2*Au+0.8*Ag")) # Example of how to add a standard
asa(DataFrame, collect(stds))

,Material,S,Se,Te,Hg,Total
,String,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…
1,HgSe,0.0,0.282482,0.0,0.717518,1.0
2,HgTe,0.0,0.0,0.388802,0.611198,1.0
3,HgS,0.137802,0.0,0.0,0.862198,1.0


The next cell performs all the useful work.  It takes an element, detector and the minimum mass-fraction and produces a table where the rows are the candidate materials and the columns are the fit regions.  A check mark indicates that the material is suitable for the region and an X indicates lack of suitability.  The ideal reference has all checks.  However, it is possible to use spectra from two or more materials to satisfy all the regions that need to be fit.

In [4]:
suitability(elm, stds, det)

,Material,Count,Hg M5-N7 + 24 others,Hg L3-M1 + 1 other,Hg L3-M5 + 2 others,Hg L2-M1,Hg L2-M4 + 18 others,Hg L2-N4 + 15 others
,String,Int64,String,String,String,String,String,String
1,HgTe,5,✗,✓,✓,✓,✓,✓
2,HgS,5,✗,✓,✓,✓,✓,✓
3,HgSe,3,✗,✓,✓,✗,✗,✓
